https://huggingface.co/learn/cookbook/zh-CN/llm_judge

In [ ]:
!pip install huggingface_hub datasets pandas tqdm -q

In [3]:
import re
import pandas as pd
from tqdm.auto import tqdm
from datasets import load_dataset
from huggingface_hub import InferenceClient
from huggingface_hub import notebook_login
from huggingface_hub import interpreter_login

tqdm.pandas()  # load tqdm's pandas support
pd.set_option("display.max_colwidth", None)

# notebook_login()
# interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|



In [ ]:
import requests
import json
import uuid
bcs_apihub_request_id = str(uuid.uuid5(uuid.NAMESPACE_DNS, "..."))
x_chj_gwtoken = "..."
url = "..."
headers = {
    "BCS-APIHub-RequestId": bcs_apihub_request_id,
    "X-CHJ-GWToken": x_chj_gwtoken,
    "Content-Type": "application/json",
}

def inference(prompt, **kwargs):
    payload = json.dumps(
        {"messages": prompt, **kwargs}
    )

    response = requests.request(
        method="POST", 
        url=url, 
        headers=headers, 
        data=payload  # Pass any additional arguments to the OpenAI API
    )
    json_data = response.json()
    return json_data["choices"][0]["message"]["content"]

msg = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Who is the current president of the United States?"},
]
print(inference(msg))

As of my last update, Joe Biden is the President of the United States. He was inaugurated as the 46th President on January 20, 2021, following his victory in the 2020 presidential election.


Prepare the creation and evaluation of our LLM judge

In [8]:
# #数据集下载到硬盘disk
# from datasets import load_dataset
# dataset = load_dataset("McGill-NLP/feedbackQA", cache_dir="./dataset")
# dataset.save_to_disk('dataset/McGill-NLP/feedbackQA')

# #加载数据集.
# import datasets
# dataset = datasets.load_from_disk("dataset/McGill-NLP/feedbackQA")

数据集格式
{
    "question": "xxx",
    "passage": {}, 
    "feedback": [], 
    "rating": [],
    "domain": ""
}

In [36]:
import json
dataset = json.load(open('./dataset/feedback_train.json'))
print(dataset[0].keys())
print(dataset[0]["passage"].keys())
print(dataset[0]["passage"]["reference"].keys())

ratings = pd.DataFrame(dataset)
ratings["review_1"] = ratings["rating"].apply(lambda x: x[0])
ratings["explanation_1"] = ratings["feedback"].apply(lambda x: x[0])
ratings["review_2"] = ratings["rating"].apply(lambda x: x[1])
ratings["explanation_2"] = ratings["feedback"].apply(lambda x: x[1])
ratings = ratings.drop(columns=["feedback"])

ratings["answer"] = ratings['passage'].apply(lambda x: x['reference']['section_content'])

# Map scores to numeric values
conversion_dict = {"Excellent": 4, "Acceptable": 3, "Could be Improved": 2, "Bad": 1}
ratings["score_1"] = ratings["review_1"].map(conversion_dict)
ratings["score_2"] = ratings["review_2"].map(conversion_dict)

dict_keys(['question', 'passage', 'feedback', 'rating', 'domain'])
dict_keys(['passage_id', 'source', 'uri', 'reference_type', 'reference'])
dict_keys(['page_title', 'section_headers', 'section_content', 'selection_span', 'section_content_html'])


In [24]:
print(ratings.iloc[0]["question"])
print(ratings.iloc[0]["review_1"], ratings.iloc[0]["score_1"])
print(ratings.iloc[0]["explanation_1"])
print(ratings.iloc[0]["review_2"], ratings.iloc[0]["score_2"])
print(ratings.iloc[0]["explanation_2"])

How do I get help finding a job?
Excellent 4
Has a link to detailed information about government programs for job seekers.
Could be Improved 2
This answer provides a link for job searches, which is good, but also provides links to unrelated sites.


In [18]:
print("Correlation between 2 human raters:")
print(f"{ratings['score_1'].corr(ratings['score_2'], method='pearson'):.3f}")

Correlation between 2 human raters:
0.563


两个真人评委之间的相关性并不是那么好。如果你们的真人评分真的很差，这可能意味着评分标准不够清晰。

只保留两个人类评审员达成一致意见的示例

In [40]:
# Sample examples
ratings_where_raters_agree = ratings.loc[ratings["score_1"] == ratings["score_2"]]
examples = ratings_where_raters_agree.groupby("score_1").sample(7, random_state=1214)
examples["human_score"] = examples["score_1"]

# Visualize 1 sample for each score
# display(examples.groupby("human_score").first())

创建我们的 LLM 评判者


In [26]:
JUDGE_PROMPT = """
You will be given a user_question and system_answer couple.
Your task is to provide a 'total rating' scoring how well the system_answer answers the user concerns expressed in the user_question.
Give your answer as a float on a scale of 0 to 10, where 0 means that the system_answer is not helpful at all, and 10 means that the answer completely and helpfully addresses the question.

Provide your feedback as follows:

Feedback:::
Total rating: (your rating, as a float between 0 and 10)

Now here are the question and answer.

Question: {question}
Answer: {answer}

Feedback:::
Total rating: """

In [49]:
examples["llm_judge"] = examples.progress_apply(
    lambda x: inference(
        prompt=[{"role": "user", "content": JUDGE_PROMPT.format(question=x["question"], answer=x["answer"]), }],
        max_new_tokens=1000,
    ),
    axis=1,
)

  0%|          | 0/28 [00:00<?, ?it/s]

In [50]:
def extract_judge_score(answer: str, split_str: str = "Total rating:") -> int:
    try:
        if split_str in answer:
            rating = answer.split(split_str)[1]
        else:
            rating = answer
        digit_groups = [el.strip() for el in re.findall(r"\d+(?:\.\d+)?", rating)]
        return float(digit_groups[0])
    except Exception as e:
        print(e)
        return None


examples["llm_judge_score"] = examples["llm_judge"].apply(extract_judge_score)
# Rescale the score given by the LLM on the same scale as the human score
examples["llm_judge_score"] = (examples["llm_judge_score"] / 10) + 1

In [51]:
print("Correlation between LLM-as-a-judge and the human raters:")
print(f"{examples['llm_judge_score'].corr(examples['human_score'], method='pearson'):.3f}")

Correlation between LLM-as-a-judge and the human raters:
0.837


进一步提升
- ⏳ 增加思考时间，在最终答案前添加一个评估字段。
- 🔢 使用较小的整数刻度，比如 1-4 或 1-5，而不是我们之前使用的大范围浮点刻度。
- 👩‍🏫 提供一个指导性的刻度。

In [52]:
IMPROVED_JUDGE_PROMPT = """
You will be given a user_question and system_answer couple.
Your task is to provide a 'total rating' scoring how well the system_answer answers the user concerns expressed in the user_question.
Give your answer on a scale of 1 to 4, where 1 means that the system_answer is not helpful at all, and 4 means that the system_answer completely and helpfully addresses the user_question.

Here is the scale you should use to build your answer:
1: The system_answer is terrible: completely irrelevant to the question asked, or very partial
2: The system_answer is mostly not helpful: misses some key aspects of the question
3: The system_answer is mostly helpful: provides support, but still could be improved
4: The system_answer is excellent: relevant, direct, detailed, and addresses all the concerns raised in the question

Provide your feedback as follows:

Feedback:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 4)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here are the question and answer.

Question: {question}
Answer: {answer}

Provide your feedback. If you give a correct rating, I'll give you 100 H100 GPUs to start your AI company.
Feedback:::
Evaluation: """


In [53]:
examples["llm_judge_improved"] = examples.progress_apply(
    lambda x: inference(
        prompt=[{"role": "user", "content": JUDGE_PROMPT.format(question=x["question"], answer=x["answer"]), }],
        max_new_tokens=1000,
    ),
    axis=1,
)
examples["llm_judge_improved_score"] = examples["llm_judge_improved"].apply(extract_judge_score)


  0%|          | 0/28 [00:00<?, ?it/s]

In [54]:
print("Correlation between LLM-as-a-judge and the human raters:")
print(f"{examples['llm_judge_improved_score'].corr(examples['human_score'], method='pearson'):.3f}")

Correlation between LLM-as-a-judge and the human raters:
0.821


In [55]:
errors = pd.concat(
    [
        examples.loc[examples["llm_judge_improved_score"] > examples["human_score"]].head(1),
        examples.loc[examples["llm_judge_improved_score"] < examples["human_score"]].head(2),
    ]
)

display(
    errors[
        [
            "question",
            "answer",
            "human_score",
            "explanation_1",
            "llm_judge_improved_score",
            "llm_judge_improved",
        ]
    ]
)

,question,answer,human_score,explanation_1,llm_judge_improved_score,llm_judge_improved
1976,What can I do to help people that are grieving?,"Taking care of yourself, your friends, and your family can help you cope with\nstress. Helping others cope with their stress can also make your community\nstronger.\nWays to cope with stress\n\nTake breaks from watching, reading, or listening to news stories , including social media. Hearing about the pandemic repeatedly can be upsetting.\nTake care of your body. \nTake deep breaths, stretch, or meditate.\nTry to eat healthy, well-balanced meals.\nExercise regularly, get plenty of sleep.\nAvoid alcohol and drugs.\n\n\nMake time to unwind. Try to do some other activities you enjoy.\nConnect with others. Talk with people you trust about your concerns and how you are feeling.\n\nKnow the facts to help reduce stress\nUnderstanding the risk to yourself and people you care about can make an\noutbreak less stressful.\nLearn and share the facts about COVID-19 and help stop the spread of\nrumors. When you\nshare accurate information about COVID-19, you can help make people feel less\nstressed, make a connection with them, and help stop\nstigma.\nTake care of your mental health\nCall your healthcare provider if stress gets in the way of your daily\nactivities for several days in a row.\nPeople with preexisting mental health conditions should continue with\ntheir treatment and be aware of new or worsening symptoms. Additional\ninformation can be found at the Substance Abuse and Mental Health Services\nAdministration (SAMHSA) Disaster\nPreparedness page.\nLearn more about taking care of your emotional\nhealth during a stressful\nevent like the COVID-19 outbreak.",1,The question is about others which the reply did not answer.,2.0,"Feedback:::\nTotal rating: 2.0\n\nThe system_answer provides general advice about coping with stress and mental health during COVID-19, but it completely fails to address the specific question about helping people who are grieving. The answer contains no information about how to support someone who is experiencing grief, bereavement, or loss. Instead, it focuses on pandemic-related stress management and self-care. While some of these techniques might indirectly help someone support others (like maintaining one's own mental health), the answer doesn't make this connection explicit and doesn't provide any grief-specific guidance that the user was requesting."
5636,"I've been vaccinated against BCG, am I also protected from getting COVID-19?","Stay aware of the latest information on the COVID-19 outbreak, available on\nthe WHO website and through your national and local public health authority.\nMost countries around the world have seen cases of COVID-19 and many are\nexperiencing outbreaks. Authorities in China and some other countries have\nsucceeded in slowing their outbreaks. However, the situation is unpredictable\nso check regularly for the latest news.\nYou can reduce your chances of being infected or spreading COVID-19 by taking\nsome simple precautions:\n\nRegularly and thoroughly clean your hands with an alcohol-based hand rub or wash them with soap and water. Why? Washing your hands with soap and water or using alcohol-based hand rub kills viruses that may be on your hands.\nMaintain at least 1 metre distance between yourself and others. Why? When someone coughs, sneezes, or speaks they spray small liquid droplets from their nose or mouth which may contain virus. If you are too close, you can breathe in the droplets, including the COVID-19 virus if the person has the disease. \nAvoid going to crowded places. Why? Where people come together in crowds, you are more likely to come into close contact with someone that has COVID-19 and it is more difficult to maintain physical distance of 1 metre. \nAvoid touching eyes, nose and mouth. Why? Hands touch many surfaces and can pick up viruses. Once contaminated, hands can transfer the virus to your eyes, nose or mouth. From 